# Notebook error process 

### filter out error notebooks with uninteresting exception types and error notebooks not using any of the top ML libraries

### 1 mark only the ones using the selected libraries -> is_MLnb


### 1.1 nbs from github

In [1]:
import pandas as pd
import util
import config

df2_err = pd.read_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'))
df2_err["is_MLnb"] = df2_err.lib_alias.apply(util.lib_alias_isML)

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [2]:
print("{0:.2%} of all the python GitHub notebooks(containing errors) use the selected ML libraries".format(sum(df2_err[["fname","is_MLnb"]].drop_duplicates().is_MLnb)/df2_err.fname.nunique()))

80.48% of all the python GitHub notebooks(containing errors) use the selected ML libraries


In [3]:
df2_err.to_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'), index=False, engine="xlsxwriter")

### 1.2 nbs from kaggle

In [4]:
import pandas as pd
import util
import config

df_err = pd.read_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'))
df_err["is_MLnb"] = df_err.lib_alias.apply(util.lib_alias_isML)

In [5]:
print("There are {0:.2%} of all error Kaggle notebooks using the selected ML libraries".format(sum(df_err[["fname","is_MLnb"]].drop_duplicates().is_MLnb)/df_err.fname.nunique()))

There are 93.45% of all error Kaggle notebooks using the selected ML libraries


In [6]:
df_err.to_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'), index=False, engine="xlsxwriter")

### 2 mark the ones with uninteresting exception types -> is_relevant

### Pending--> 2.1 First, let us gather all the exception types appeared in both error dataset and select a list of uninteresting ones:

    https://liuonline-my.sharepoint.com/:x:/r/personal/yirwa29_liu_se/_layouts/15/Doc.aspx?sourcedoc=%7B55EB4974-57AF-46DF-A27E-83FEBAB67B69%7D&file=nberror_exception_types.xlsx&action=default&mobileredirect=true

    Write to **config.builtin_exps_excluded** when selected.

In [2]:
import pandas as pd
import util
import config

df2_err = pd.read_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'))
df_err = pd.read_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'))

In [23]:
# df_tmp1 = df_err[["ename", "ename_mapped"]].drop_duplicates().sort_values(by=['ename_mapped'])
# df_tmp2 = df2_err[["ename", "ename_mapped"]].drop_duplicates().sort_values(by=['ename_mapped'])
# df_tmp3 = pd.merge(df_tmp1, df_tmp2, on="ename", how="outer")
# df_tmp3['ename_mapped'] = df_tmp3['ename_mapped_x'].fillna(df_tmp3['ename_mapped_y'])
# df_tmp3 = df_tmp3[["ename", "ename_mapped"]]
# df_tmp3.to_excel(config.path_default + '/nberror_exception_types.xlsx', index=False, engine="xlsxwriter")

### 2.2 Mark if relevant interms of exceptions for nbs from kaggle and github

#### ----> we now only exclude "keyboardinterrupt"

In [3]:
df_err["is_relevant"] = ~df_err.ename_mapped.isin(config.builtin_exps_excluded)
df2_err["is_relevant"] = ~df2_err.ename_mapped.isin(config.builtin_exps_excluded)

In [4]:
print("There are {0:.2%} of all errors in Kaggle notebooks have relevant exception types".format(sum(df_err.is_relevant)/len(df_err)))
print("There are {0:.2%} of all errors in GitHub notebooks have relevant exception types".format(sum(df2_err.is_relevant)/len(df2_err)))

There are 83.28% of all errors in Kaggle notebooks have relevant exception types
There are 89.41% of all errors in GitHub notebooks have relevant exception types


In [5]:
df_err.to_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'), index=False, engine="xlsxwriter")
df2_err.to_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'), index=False, engine="xlsxwriter")

### 3 Statistics if filtering out all the errors that are not from ML notebooks and all errors that are not relevant exceptions

In [1]:
import pandas as pd
import util
import config

df2_err = pd.read_excel(config.path_github_error_process.joinpath('nberror_g_all_eid_p.xlsx'))
df_err = pd.read_excel(config.path_kaggle_error_process.joinpath('nberror_k_eid_p.xlsx'))

path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [20]:
def print_statistics(df_tar_all, category="Kaggle", if_py3=True):
    if if_py3:
        df_tar = df_tar_all[df_tar_all.python_version==3]
    else:
        df_tar = df_tar_all
    print("{:.2%}({}) of all errors({}),{:.2%}({}) of all notebooks({}), in {} notebooks use the selected ML libraries".format(
        len(df_tar[df_tar.is_MLnb])/len(df_tar),
        len(df_tar[df_tar.is_MLnb]),
        len(df_tar),
        df_tar[df_tar.is_MLnb].fname.nunique()/df_tar.fname.nunique(),
        df_tar[df_tar.is_MLnb].fname.nunique(),
        df_tar.fname.nunique(),
        category))
    print("{:.2%}({}) of all errors({}),{:.2%}({}) of all notebooks({}), in {} notebooks have relevant exception types".format(
        len(df_tar[df_tar.is_relevant])/len(df_tar),
        len(df_tar[df_tar.is_relevant]),
        len(df_tar),
        df_tar[df_tar.is_relevant].fname.nunique()/df_tar.fname.nunique(),
        df_tar[df_tar.is_relevant].fname.nunique(), 
        df_tar.fname.nunique(),
        category))
    print("{:.2%}({}) of all errors({}),{:.2%}({}) of all notebooks({}), in {} notebooks can be parsed into AST".format(
        len(df_tar[df_tar.if_ast==1.0])/len(df_tar),
        len(df_tar[df_tar.if_ast==1.0]),
        len(df_tar),
        df_tar[df_tar.if_ast==1.0].fname.nunique()/df_tar.fname.nunique(),
        df_tar[df_tar.if_ast==1.0].fname.nunique(), 
        df_tar.fname.nunique(),
        category))
    df_tar_filtered = df_tar[df_tar.is_MLnb&df_tar.is_relevant&df_tar.if_ast==1.0]
    print("{:.2%}({}) of all errors({}),{:.2%}({}) of all notebooks({}), in {} notebooks remain, after all the filtering above.".format(
        len(df_tar_filtered)/len(df_tar), 
        len(df_tar_filtered), 
        len(df_tar),
        df_tar_filtered.fname.nunique()/df_tar.fname.nunique(),
        df_tar_filtered.fname.nunique(),
        df_tar.fname.nunique(),
        category))
    print("\n")

print_statistics(df_err, category="Kaggle", if_py3=False)
print_statistics(df2_err, category="GitHub", if_py3=False)
print("If only consider notebooks with python version 3\n")
print_statistics(df_err, category="Kaggle")
print_statistics(df2_err, category="GitHub")

91.53%(6202) of all errors(6776),93.45%(4064) of all notebooks(4349), in Kaggle notebooks use the selected ML libraries
83.28%(5643) of all errors(6776),82.62%(3593) of all notebooks(4349), in Kaggle notebooks have relevant exception types
77.11%(5225) of all errors(6776),82.18%(3574) of all notebooks(4349), in Kaggle notebooks can be parsed into AST
57.87%(3921) of all errors(6776),62.22%(2706) of all notebooks(4349), in Kaggle notebooks remain, after all the filtering above.


77.89%(138075) of all errors(177258),80.48%(87665) of all notebooks(108925), in GitHub notebooks use the selected ML libraries
89.41%(158489) of all errors(177258),88.25%(96123) of all notebooks(108925), in GitHub notebooks have relevant exception types
82.09%(145516) of all errors(177258),87.16%(94939) of all notebooks(108925), in GitHub notebooks can be parsed into AST
57.28%(101532) of all errors(177258),61.30%(66771) of all notebooks(108925), in GitHub notebooks remain, after all the filtering above.


If o

In [41]:
# can be presented in stacked chart, 
# e.g., is_relevant (is_ML, ~is_ML), ~is_relevant (is_ML, ~is_ML)
# e.g., is_ast (is_ML, ~is_ML), ~is_ast(is_ML, ~is_ML)